<a href="https://colab.research.google.com/github/rmwvamp/AStartPathfinding/blob/main/cifar_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

In [28]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training")


[1,   200] loss: 2.303
[1,   400] loss: 2.303
[1,   600] loss: 2.302
[2,   200] loss: 2.300
[2,   400] loss: 2.298
[2,   600] loss: 2.296
[3,   200] loss: 2.278
[3,   400] loss: 2.237
[3,   600] loss: 2.133
[4,   200] loss: 1.987
[4,   400] loss: 1.951
[4,   600] loss: 1.910
[5,   200] loss: 1.807
[5,   400] loss: 1.769
[5,   600] loss: 1.709
[6,   200] loss: 1.638
[6,   400] loss: 1.611
[6,   600] loss: 1.593
[7,   200] loss: 1.561
[7,   400] loss: 1.526
[7,   600] loss: 1.516
[8,   200] loss: 1.483
[8,   400] loss: 1.455
[8,   600] loss: 1.461
[9,   200] loss: 1.409
[9,   400] loss: 1.405
[9,   600] loss: 1.399
[10,   200] loss: 1.364
[10,   400] loss: 1.350
[10,   600] loss: 1.351
Finished Training


In [32]:
from sklearn.metrics import confusion_matrix, classification_report


all_preds = []
all_labels = []


net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)


        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


conf_matrix = confusion_matrix(all_labels, all_preds)


print("Matrix:")
print(conf_matrix)


class_names = classes
classification_rep = classification_report(all_labels, all_preds, target_names=class_names)
print("\nClassification Report:")
print(classification_rep)


Confusion Matrix:
[[580  14  73  35  20  18  22  16 156  66]
 [ 56 454   9  33   2  12  17  15 109 293]
 [ 71   7 363 129  85 147  92  49  25  32]
 [ 16   4  60 410  28 287 105  42  16  32]
 [ 41   5 176  95 278 109 164  98  20  14]
 [ 11   2  77 183  20 580  44  59  11  13]
 [  3   2  61 113  49  56 661  22  10  23]
 [ 26   1  31  89  41 181  37 552   5  37]
 [147  27  17  26   3  21  13   7 667  72]
 [ 42  64  15  60   3  24  34  36  99 623]]

Classification Report:
              precision    recall  f1-score   support

       plane       0.58      0.58      0.58      1000
         car       0.78      0.45      0.57      1000
        bird       0.41      0.36      0.39      1000
         cat       0.35      0.41      0.38      1000
        deer       0.53      0.28      0.36      1000
         dog       0.40      0.58      0.48      1000
        frog       0.56      0.66      0.60      1000
       horse       0.62      0.55      0.58      1000
        ship       0.60      0.67      0